# Example using eeMLPRegressor / eeStandardScaler
In this short example, we show how a locally trained MLPRegressor from the library scikit-learn can be used for server-side predictions using Google Earth Engine (GEE). Note, that the library is work in progress and has not been sufficiently tested so far. 

In [12]:
import os
import ee
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from eetranslator.eeMLPRegressor import eeMLPRegressor
from eetranslator.eeStandardScaler import eeStandardScaler

In [13]:
ee.Initialize()

## Load Data and Train Model locally

In [14]:
# Load training data: consists of sentinel-2 bands and vegetation trait variables 
# training data created using prosail rtm model https://jbferet.gitlab.io/prosail/index.html 
path = os.path.join('..', 'data', 'rtm_s2.csv')

dat = pd.read_csv(path)
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']
target = 'CHL'

X, y = dat[bands], dat[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [15]:
# a common practice is to standardize the data before training
X_scaler = StandardScaler()
y_scaler = StandardScaler()

X_scaler.fit(X_train)
y_scaler.fit(y_train.values.reshape(-1, 1))

X_train_scaled, X_test_scaled = X_scaler.transform(X_train), X_scaler.transform(X_test)
y_train_scaled, y_test_scaled = y_scaler.transform(y_train.values.reshape(-1, 1)), y_scaler.transform(y_test.values.reshape(-1, 1))

In [16]:
# train a neural network model
model = MLPRegressor(hidden_layer_sizes=(50, 50), max_iter=1000, random_state=42)
model.fit(X_train_scaled, y_train_scaled)

# evaluate the model
print('Training score: {}'.format(model.score(X_train_scaled, y_train_scaled)))
print('Test score: {}'.format(model.score(X_test_scaled, y_test_scaled)))

Training score: 0.9279953392881533
Test score: 0.8914593492875191


## Convert local model to perform GEE server-side predictions

In [17]:
# load an example S2 image from Google Earth Engine

image = ee.Image('COPERNICUS/S2_SR_HARMONIZED/20220718T102559_20220718T103413_T32TMT').select(bands).divide(10000)

In [18]:
ee_model = eeMLPRegressor(model, prediction_name='CHL')
ee_X_scaler = eeStandardScaler(X_scaler, feature_names=bands)
ee_y_scaler = eeStandardScaler(y_scaler, feature_names=[target]) # feature_names must be a list

In [19]:
image_scaled = ee_X_scaler.transform_image(image)
predictions_scaled = ee_model.classify(image_scaled)
predictions = ee_y_scaler.inverse_transform_column(predictions_scaled, column='CHL')

In [20]:
predictions.getInfo()

{'type': 'Image',
 'bands': [{'id': 'CHL',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'dimensions': [10980, 10980],
   'crs': 'EPSG:32632',
   'crs_transform': [10, 0, 399960, 0, -10, 5300040]}]}

## Visualize results

In [21]:
# visualize the results using geemap
import geemap
Map = geemap.Map()
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}, 'S2')
Map.addLayer(predictions, {'min': 0, 'max': 100}, 'CHL')
Map.centerObject(image.geometry())
# show the map
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…